In [2]:
import seaborn as sns
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor
from sklearn.preprocessing import PolynomialFeatures
from IPython.display import display, HTML

#import my modules
import acquire as a
import prepare as p
import wrangle_zillow as w

from sklearn.preprocessing import MinMaxScaler, RobustScaler

# Acquire

In [ ]:
df = a.get_zillow()

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
#getting the latest transactions 
df = df.sort_values(by ='transactiondate', ascending=True).drop_duplicates( subset = 'parcelid' ,keep= 'last')
  

In [ ]:
df.shape

In [ ]:
#this list has all types of single unit properties
single= ['Single Family Residential',' Mobile Home' , 'Townhouse ', 'Manufactured, Modular, Prefabricated Homes'  ]

In [ ]:
#create a mask
single_mask = df['propertylandusedesc'].isin(single)

In [ ]:
#if unitcnt == that's means is is a single property but if it is null we can compare with our list of single

In [ ]:
#using that mask and also add  a condition
df_single = df[single_mask & ((df['unitcnt'] == 1) | (df['unitcnt'].isnull()))]
    

In [ ]:
#this df contains all single unit homes of 2017 with the latest transaction
df_single.shape

In [ ]:
a.summarize(df_single)

**takeaways**
- numberofstories there is one property with 6 stories , I would drop it!
- unitcnt is not needed any more. I would drop it
- there are a lot of nulls so in preparation step I will handle them

# Preparation

In [ ]:
p.miss_dup_values(df_single)

In [ ]:
df_single.shape

In [ ]:
# I will drop columns and rows with a lot of null values 
#here I will include unitcnt because the value is 1 for all the properties . as it is the next to 65.6% 
#I can include it in my function
df_single= p.handle_missing_values(df_single, prop_required_columns=0.75, prop_required_row=0.75)

In [ ]:
df_single.shape

In [ ]:
p.miss_dup_values(df_single)

In [ ]:

#all these  columns represent a minimun amount so I can drop them 
df_single= p.drop_low_missing_values(df_single, per= 3 )

In [ ]:
df_single.shape

In [ ]:
p.miss_dup_values(df_single)

In [ ]:
a.summarize(df_single)

In [ ]:
p.distribution(df_single)

### get county names and create dummy variables with those

In [ ]:
# create dummy vars of fips id
county_df = pd.get_dummies(df_single.fips)

In [ ]:
county_df.head()

- county # 6037 -----> Los Angeles
- county # 6059 -----> Orange
- county # 6111 -----> Ventura

In [ ]:
df_single = p.get_counties(df_single)

In [ ]:
# print("LA County Verified: ", df_single[df_single.fips==6037]['fips'].count() == df_single.los_angeles.sum())
# print("Orange County Verified: ", df_single[df_single.fips==6059]['fips'].count() == df_single.orange.sum())
# print("Ventura County Verified: ", df_single[df_single.fips==6111]['fips'].count() == df_single.ventura.sum())

In [ ]:
#now in my function I can drop fips

###  Compute new features out of existing features in order to reduce noise, capture signals, and reduce collinearity, or dependence between independent variables.

**- age**

In [ ]:
df_single['age'] = 2017 - df.yearbuilt

In [ ]:
df_single.head()

In [ ]:
df_single[['age']].describe()

In [ ]:
# I can drop yearbuilt

**- taxrate**


In [ ]:
df_single['taxrate'] = df_single.taxamount/df_single.taxvaluedollarcnt*100

In [ ]:
df_single[['taxrate']].describe()

In [ ]:
# I will drop taxamount and taxvaluedollarcnt

**- transactiondate**


In [ ]:
df_single[['transactiondate']].head()

In [ ]:
#I decided to convert transaction date to int so in the future can make groups

In [ ]:
df_single['transactiondate']=(df_single['transactiondate'].str.replace(' ','').str.replace('-',''))

In [ ]:
df_single['transactiondate'] = df_single['transactiondate'].astype('int')

In [ ]:
#df_single[(df_single.transactiondate >= 20170101) &  (df_single.transactiondate < 20170501)]

In [ ]:
df_single.info()

In [ ]:
df_single['lotsize_acres'] = df_single.lotsizesquarefeet/43560
df_single = df_single.drop(columns = ['yearbuilt', 'taxamount', 'taxvaluedollarcnt', 'lotsizesquarefeet'  ])

In [ ]:
def create_features (df) :
    '''
    takes in a df and create age , taxrate, lotsize_acres columns and convert transactiondate to int
    drops 'yearbuilt', 'taxamount', 'taxvaluedollarcnt', lotsizesquarefeet columns
    '''
    #create a new colum with age
    df['age'] = 2017 - df.yearbuilt
    
    #taxrate
    df['taxrate'] = df.taxamount/df.taxvaluedollarcnt*100
    
    #transactiondate
    df['transactiondate']=(df['transactiondate'].str.replace(' ','').str.replace('-',''))
    df['transactiondate'] = df['transactiondate'].astype('int')
    
     # create acres variable
    df['lotsize_acres'] = df.lotsizesquarefeet/43560
    
    #drop columns
    df = df.drop(columns = ['yearbuilt', 'taxamount', 'taxvaluedollarcnt', 'lotsizesquarefeet'  ])
    
    return df

In [ ]:
p.distribution_boxplot(df_single)

In [ ]:
df_single.columns

In [ ]:
# I will handle outliers for : 'bathroomcnt', 'bedroomcnt', 'calculatedfinishedsquarefeet', 'regionidzip', 
#'structuretaxvaluedollarcnt', 'landtaxvaluedollarcnt', 'taxrate', 'lotsize_acres', 'age'

In [ ]:
col = ['bathroomcnt', 'bedroomcnt', 'calculatedfinishedsquarefeet', 'regionidzip', 'structuretaxvaluedollarcnt', 'landtaxvaluedollarcnt', 'taxrate', 'lotsize_acres', 'age']

In [ ]:
df_final= p.remove_outliers(df_single, col)

In [ ]:
p.distribution_boxplot(df_final)

In [ ]:
df_final.shape, df_single.shape

In [ ]:
df_final['regionidcity'].value_counts()

In [ ]:
df_final.info()

In [ ]:
# here I can drop
# propertylandusedesc 

In [ ]:
#try to bin transaction date
df_final['quadrimester'] = pd.cut(df_final.transactiondate, bins = [ 20170100, 20170500, 20170900, 20171230])

In [ ]:
df_final['quadrimester'].value_counts()

In [ ]:
#this is just to check if bins are correct
#this is the fist qua
(df_final['transactiondate']<20170501).sum()

In [ ]:
((df_final['transactiondate']>= 20170501)  & (df_final['transactiondate']<20170901)).sum()

In [ ]:
((df_final['transactiondate']>= 20170901)  & (df_final['transactiondate']<20171232)).sum()

In [ ]:
#try to bin transaction date
df_final['quadrimester'] = pd.cut(df_final.transactiondate, bins = [ 20170100, 20170500, 20170900, 20171230],
                                 labels = [1,2,3])

In [ ]:
df_final['quadrimester'].value_counts()

In [ ]:
df_final[['transactiondate', 'quadrimester']].sample(30)

In [3]:
#checking wrangle_zillow function
train, validate, test= w.wrangle_zillow()

before outliers (50819, 21)
df shape --> (34126, 20)
train -> (19110, 20)
validate -> (8190, 20)
test -> (6826, 20)


In [5]:
((50819-34126)*100) / 50819

32.847950569668825